In [1]:
!pip install transformers[onnx]
!pip install git+https://github.com/openai/whisper.git 
!pip install onnx
!pip install onnx_tf
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0_thg601
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-0_thg601
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python -m transformers.onnx --help

usage: Hugging Face Transformers ONNX exporter [-h] -m MODEL
                                               [--feature {causal-lm,causal-lm-with-past,default,default-with-past,image-classification,image-segmentation,masked-im,masked-lm,multiple-choice,object-detection,question-answering,semantic-segmentation,seq2seq-lm,seq2seq-lm-with-past,sequence-classification,speech2seq-lm,speech2seq-lm-with-past,token-classification,vision2seq-lm}]
                                               [--opset OPSET] [--atol ATOL]
                                               [--framework {pt,tf}]
                                               [--cache_dir CACHE_DIR]
                                               [--preprocessor {auto,tokenizer,feature_extractor,processor}]
                                               output

positional arguments:
  output                Path indicating where to store generated ONNX model.

optional arguments:
  -h, --help            show this help message and exit
  

In [3]:
!python -m transformers.onnx --model=openai/whisper-tiny.en onnx/

Framework not requested. Using torch to export to ONNX.
Using framework PyTorch: 1.12.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> False
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
/usr/local/lib/python3.7/dist-packages/transformers/models/whisper/modeling_whisper.py:200: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.7/dist-packages/transformers/models/whisper/modeling_whisper.py:239: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as

In [4]:
!git clone https://github.com/openai/whisper.git

fatal: destination path 'whisper' already exists and is not an empty directory.


In [5]:
import whisper
import torch
import tensorflow as tf
import onnx
import numpy as np
import argparse
import os
import warnings
import tqdm
from onnx_tf.backend import prepare
from whisper.audio import load_audio, log_mel_spectrogram,pad_or_trim,N_FRAMES, SAMPLE_RATE

tf_model_path = 'whisper_model_tf'
onnx_model = onnx.load("/content/onnx/model.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [6]:
import torch
import tensorflow as tf
from transformers import TFWhisperModel, WhisperFeatureExtractor
from datasets import load_dataset

model = TFWhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
inputs = feature_extractor(ds[0]["audio"]["array"], return_tensors="tf")
input_features = inputs.input_features
decoder_input_ids = tf.convert_to_tensor([[1, 1]]) * model.config.decoder_start_token_id
last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state
list(last_hidden_state.shape)

All model checkpoint layers were used when initializing TFWhisperModel.

All the layers of TFWhisperModel were initialized from the model checkpoint at openai/whisper-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperModel for predictions without further training.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[1, 2, 512]

In [7]:
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

saved_model_dir = 'whisper_model_tf'
tflite_model_path = 'whisper-int8.tflite'

def representative_dataset():
    for x in range(20):
      inputs = feature_extractor(ds[x]["audio"]["array"],sampling_rate=16000, return_tensors="tf")
      input_features = inputs.input_features
      decoder_input_ids = tf.convert_to_tensor([[1, 1]]) * model.config.decoder_start_token_id
      decoder_input_ids = np.array(decoder_input_ids, dtype=np.int64)
      yield [decoder_input_ids, input_features.astype(np.float32)]

# Convert to tflite(int8) model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.representative_dataset = representative_dataset
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
import tensorflow as tf
import numpy as np
tflite_model_path = '/content/whisper-encoder-int8.tflite'

In [8]:
%ls -la

total 37440
drwxr-xr-x 1 root root     4096 Nov  2 21:13 ./
drwxr-xr-x 1 root root     4096 Nov  2 17:48 ../
drwxr-xr-x 4 root root     4096 Oct 28 13:36 .config/
drwxr-xr-x 2 root root     4096 Nov  2 21:13 .ipynb_checkpoints/
drwxr-xr-x 2 root root     4096 Nov  2 17:49 onnx/
drwxr-xr-x 1 root root     4096 Oct 28 13:37 sample_data/
drwxr-xr-x 7 root root     4096 Nov  2 17:50 whisper/
-rw-r--r-- 1 root root 38305296 Nov  2 21:25 whisper-int8.tflite
drwxr-xr-x 4 root root     4096 Nov  2 21:22 whisper_model_tf/
